In [1]:
import utils as utils
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joshuasegal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# dataset_path = 'dataset'
# papers_path = 'papers'
# presentations_path = 'presentations'
#
# utils.move_xml_files(dataset_path, papers_path, presentations_path)

Moved XML files from 1069 folder.
Moved XML files from 1855 folder.
Moved XML files from 797 folder.
Moved XML files from 4729 folder.
Moved XML files from 3954 folder.
Moved XML files from 2886 folder.
Moved XML files from 3168 folder.
Moved XML files from 4943 folder.
Moved XML files from 2288 folder.
Moved XML files from 909 folder.
Moved XML files from 2872 folder.
Moved XML files from 135 folder.
Moved XML files from 307 folder.
Moved XML files from 2618 folder.
Moved XML files from 763 folder.
Moved XML files from 551 folder.
Moved XML files from 3365 folder.
Moved XML files from 1432 folder.
Moved XML files from 4140 folder.
Moved XML files from 1600 folder.
Moved XML files from 4372 folder.
Moved XML files from 3157 folder.
Moved XML files from 2085 folder.
Moved XML files from 1264 folder.
Moved XML files from 4716 folder.
Moved XML files from 61 folder.
Moved XML files from 3533 folder.
Moved XML files from 3701 folder.
Moved XML files from 1056 folder.
Moved XML files from 4

In [ ]:
sample_xml_pres_path = "sample_data/presentations/slide.clean_tika.xml"
sample_xml_pres = utils.read_file(sample_xml_pres_path)
sample_xml_pres

sample_xml_paper_path = "sample_data/papers/Paper_BRM.tei.xml"
sample_xml_paper = utils.read_file(sample_xml_paper_path)
sample_xml_paper

In [ ]:
sample_pres_text = utils.parse_presentation_xml(sample_xml_pres)
sample_pres_text[0]

In [ ]:
sample_paper_text = utils.parse_paper_xml(sample_xml_paper)
sample_paper_text[0]

In [ ]:
sample_pres_preprocessed = utils.preprocess_text(sample_pres_text)

In [ ]:
sample_paper_preprocessed = utils.preprocess_text(sample_paper_text)

In [ ]:
print(sample_pres_preprocessed[0])
print(sample_paper_preprocessed[0])

In [ ]:
presentation_word_model = Word2Vec(sentences = sample_pres_preprocessed, vector_size = 50, window = 5, min_count = 1, workers = 3, sg = 1)
paper_word_model = Word2Vec(sentences = sample_paper_preprocessed, vector_size = 50, window = 5, min_count = 1, workers = 3, sg = 1)

In [ ]:
print(presentation_word_model)
print(paper_word_model)

In [ ]:
EMBEDDING_PRES_MODEL_FILE = "pres_word_model.txt"
EMBEDDING_PAPER_MODEL_FILE = "paper_word_model.txt"

presentation_word_model.wv.save_word2vec_format(EMBEDDING_PRES_MODEL_FILE, binary=False)
paper_word_model.wv.save_word2vec_format(EMBEDDING_PAPER_MODEL_FILE, binary=False)

In [ ]:
presentation_folder = "dataset/presentations"
paper_folder = "dataset/papers"

# presentation_to_paper = utils.create_presentation_to_paper_mapping(presentation_folder, paper_folder)
#
# presentations_data = utils.process_presentation_folder(presentation_folder)
# papers_data = utils.process_papers_folder(paper_folder)



In [ ]:
# ## check if works
#
# word_tokenizer = Tokenizer()
# word_tokenizer.fit_on_texts(presentations_data)
# encoded = word_tokenizer.texts_to_sequences(presentations_data)
#
# char_tokenizer = Tokenizer()
# char_tokenizer.fit_on_texts(papers_data)
# encoded = char_tokenizer.texts_to_sequences(papers_data)

In [ ]:
# ## correct implementation for LR ?
#
# word_map, index_map = utils.read_embeddings("spooky_embedding_word.txt", word_tokenizer)
# char_map, char_index_map = utils.read_embeddings("spooky_embedding_char.txt", char_tokenizer)

In [ ]:
# tfidf_vectorizer = TfidfVectorizer()
# X = tfidf_vectorizer.fit_transform(presentations_data)
#
# y = list(range(len(presentations_data)))
#
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#
# logreg_model = LogisticRegression(max_iter=1000)
# logreg_model.fit(X_train, y_train)
#
# y_pred = logreg_model.predict(X_test)
#
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [ ]:
# # print(y_pred[0])
# # print(y_test[0])
# # print(presentations_data)
# print(X_train)
# print(X_test)
# # print(len(y))